<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Cross Validation - Decision Tree - Titanic
</b></font> </br></p>

---


# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
from pandas import read_excel, DataFrame
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import (
    StratifiedKFold,
    KFold,
    LeaveOneOut,
    cross_validate
)
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

import plotly.express as px
import plotly.subplots as sp

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1  | Understand
***

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
📒 Anwendungsfall
</font></p>

Dies ist der legendäre Titanic ML-Wettbewerb – die beste erste Herausforderung, um in ML-Modellierung einzutauchen.

Die Aufgabe ist einfach: Verwenden Sie maschinelles Lernen, um ein Modell zu erstellen, das vorhersagt, welche Passagiere den Schiffbruch der Titanic überlebt haben.

[Titanic Org](https://www.encyclopedia-titanica.org/)

[DataSet](https://www.openml.org/search?type=data&status=active&id=41265)

[Info](https://www.kaggle.com/competitions/titanic/data)



**Datenfelder:**   
+ Age: Alter
+ Fare: Ticketpreis
+ Sex: Geschlecht (0 = männlich, 1 = weiblich)
+ sibsp: Der Datensatz definiert Familienbeziehungen auf diese Weise ... Geschwister = Bruder, Schwester, Stiefbruder, Stiefschwester Ehepartner = Ehemann, Ehefrau (Geliebte und Verlobte wurden ignoriert)
+ parch: Der Datensatz definiert Familienbeziehungen auf diese Weise ... Elternteil = Mutter, Vater Kind = Tochter, Sohn, Stieftochter, Stiefsohn. Einige Kinder reisten nur mit einem Kindermädchen, daher ist für sie Parch=0
+ Pclass: Passagierklasse, 1.- 3. Klasse
+ Embarked: Hafen der Einschiffung

In [ ]:
df = read_excel(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/titanic.xlsx",
    usecols=["pclass", "survived", "sex", "age", "sibsp", "parch"],
)

In [ ]:
data = df.copy()
target = data.pop("survived")

<p><font color='black' size="5">
🔎 EDA (Exploratory Data Analysis) mit Pandas
</font></p>

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.groupby("sex").count()

In [ ]:
target.value_counts()

#  2 | Prepare

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Train-Test-Split durchführen</br>
✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>


<font color='black' size="5">
Datentyp ermitteln
</font>

In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<font color='black' size="5">
Train-Test-Set
</font>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.20, stratify=target, random_state=42)

<font color='black' size="5">
Missing Values - kategoriale Features
</font>

In [ ]:
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit(data_train[cat_col])
data_train[cat_col] = imputer_cat.transform(data_train[cat_col])
data_test[cat_col] = imputer_cat.transform(data_test[cat_col])

<font color='black' size="5">
Kodierung
</font>

In [ ]:
encoder = OrdinalEncoder()
encoder.fit(data_train[cat_col])
data_train[cat_col] = encoder.transform(data_train[cat_col])
data_test[cat_col] = encoder.transform(data_test[cat_col])

<font color='black' size="5">
Missing Values - numerische Features
</font>

In [ ]:
imputer_num = SimpleImputer(strategy="mean")
imputer_num.fit(data_train[num_col])
data_train[num_col] = imputer_num.transform(data_train[num_col])
data_test[num_col] = imputer_num.transform(data_test[num_col])

<font color='black' size="5">
Skalierung
</font>

In [ ]:
scaler = MinMaxScaler()
scaler.fit(data_train[all_col])
data_train[all_col] = scaler.transform(data_train[all_col])
data_test[all_col] = scaler.transform(data_test[all_col])

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

<p><font color='black' size="5">
Modellauswahl & Training
</font></p>

In [ ]:
model = DecisionTreeClassifier(max_depth=3)

In [ ]:
model.fit(data_train, target_train)

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Prognose
</font></p>

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

<p><font color='black' size="5">
Confusion Matrix
</font></p>

In [ ]:
conf_matrix = confusion_matrix(target_test, target_test_pred)
display_labels_ = ["Not Survived", "Survived"]
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=display_labels_)
disp.plot(cmap="Blues")

In [ ]:
print(
    classification_report(target_test, target_test_pred, target_names=display_labels_)
)

<p><font color='black' size="5">
Accuracy
</font></p>

In [ ]:
acc_train = accuracy_score(target_train, target_train_pred) * 100
print(f"Modell: {model} -- Train -- Accuracy: {acc_train:5.2f}")

In [ ]:
acc_test = accuracy_score(target_test, target_test_pred) * 100
print(f"Modell: {model} -- Test -- Accuracy: {acc_test:5.2f}%")

<p><font size="5">
Feature Importance
</p>

In [ ]:
title_ = "Feature Importance Titanic"
px.bar(
    x=model.feature_importances_, y=data.columns, title=title_, width=800, height=600
).update_yaxes(categoryorder="total ascending")

<p><font size="5">
Cross-Validation - K-Fold
</p>



In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
cv_results = cross_validate(
    model,
    data_train,
    target_train,
    scoring="accuracy",
    cv=cv,
    return_train_score=True,
    return_estimator=True,
)

In [ ]:
cv_results

In [ ]:
train_result = cv_results["train_score"].mean()
val_result = cv_results["test_score"].mean()
print(f"Train {train_result:.2f} -- Validation {val_result:.2f}")

<p><font size="5">
Datensätze der einzelnen Kreuzvalidierungen
</p>

In [ ]:
cv.get_n_splits(data_train)
print(cv)

for i, (train_index, val_index) in enumerate(cv.split(data_train, target_train)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index[:10]}, count: {len(train_index):,.0f}")
    print(f"  Test:  index={val_index[:10]}, count: {len(val_index):,.0f}")

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>